# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable

# YOUR CODE HERE!
load_dotenv()
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
api_key = os.getenv('NEWS_API_KEY')

In [3]:
# Check api keys
print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(api_key))

<class 'str'>
<class 'str'>
<class 'str'>


In [4]:
# Create a newsapi client
# YOUR CODE HERE!
newsapi = NewsApiClient(api_key=api_key)

In [5]:
# Fetch the Bitcoin news articles
# YOUR CODE HERE!
bitcoin_news_en = newsapi.get_everything(
    q="bitcoin",
    language="en"
)

# Show the total number of news
bitcoin_news_en["totalResults"]

7288

In [6]:
# Fetch the Ethereum news articles
# YOUR CODE HERE!
ethereum_news_en = newsapi.get_everything(
    q="ethereum",
    language="en"
)

# Show the total number of news
ethereum_news_en["totalResults"]

3544

In [7]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!

#Function to create the bitcoin and ethereum dataframes
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError as ae:
            pass

    return pd.DataFrame(articles)

In [8]:
# Bitcoin sentiment scores Dataframe
bitcoin_en_df = create_df(bitcoin_news_en["articles"], "en")

# Ethereum sentiment scores DataFrame
ethereum_en_df = create_df(ethereum_news_en["articles"], "en")

In [9]:
""" Create sentiment score function """

""" BITCOIN """

def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

# Sentiment scores dictionaries
title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    "title_sent": [],
}
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text and the title
for index, row in bitcoin_en_df.iterrows():
    try:
        # Sentiment scoring with VADER
        title_sentiment = analyzer.polarity_scores(row["title"])
        title_sent["title_compound"].append(title_sentiment["compound"])
        title_sent["title_pos"].append(title_sentiment["pos"])
        title_sent["title_neu"].append(title_sentiment["neu"])
        title_sent["title_neg"].append(title_sentiment["neg"])
        title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
title_sentiment_df = pd.DataFrame(title_sent)
text_sentiment_df = pd.DataFrame(text_sent)
bitcoin_en_df = bitcoin_en_df.join(title_sentiment_df).join(text_sentiment_df)

In [10]:
""" ETHEREUM """

# Sentiment scores dictionaries
title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    "title_sent": [],
}
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text and the title
for index, row in ethereum_en_df.iterrows():
    try:
        # Sentiment scoring with VADER
        title_sentiment = analyzer.polarity_scores(row["title"])
        title_sent["title_compound"].append(title_sentiment["compound"])
        title_sent["title_pos"].append(title_sentiment["pos"])
        title_sent["title_neu"].append(title_sentiment["neu"])
        title_sent["title_neg"].append(title_sentiment["neg"])
        title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
title_sentiment_df = pd.DataFrame(title_sent)
text_sentiment_df = pd.DataFrame(text_sent)
ethereum_en_df = ethereum_en_df.join(title_sentiment_df).join(text_sentiment_df)

In [39]:
bitcoin_en_df['text'][0]

'When Russia invaded Ukraine, Niki Proshin was already a year into making a living as a vlogger — he had a YouTube channel, a TikTok channel, and an Instagram. He also ran an online Russian club for a… [+5883 chars]'

In [12]:
ethereum_en_df.head()

,title,description,text,date,language,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
0,Web3 Threatens to Segregate Our Online Lives,Governance tokens seem like a tantalizing solu...,"In February, shit hit the fan in the usual way...",2022-03-01,en,-0.3818,0.0,0.698,0.302,-1,-0.3182,0.059,0.848,0.093,-1
1,Coinbase earnings show trading of ethereum and...,Ethereum trading volume increased from 15% to ...,Coinbase reported that the share of trading vo...,2022-02-25,en,0.0000,0.0,1.000,0.000,0,0.6705,0.188,0.812,0.000,1
2,How Ukrainians are fundraising in cryptocurrency,Millions of dollars of cryptocurrency have flo...,Illustration by James Bareham / The Verge\r\n\...,2022-02-26,en,0.0000,0.0,1.000,0.000,0,-0.4588,0.000,0.917,0.083,-1
3,How People Actually Make Money From Cryptocurr...,Power traders use “staking” and “yield farming...,"If it sounds too good to be true, youre not wr...",2022-03-13,en,0.0000,0.0,1.000,0.000,0,0.8340,0.236,0.713,0.050,1
4,What You Need to Know About Ethereum's Role in...,This now-seven-year-old decentralized and open...,"It seems that in 2022, you cant escape from th...",2022-03-03,en,0.0000,0.0,1.000,0.000,0,-0.1326,0.000,0.956,0.044,-1


In [13]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!
bitcoin_en_df.describe()

,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000
mean,-0.026690,0.073750,0.829800,0.096400,0.000000,-0.028205,0.047650,0.902150,0.050100,-0.10000
std,0.260131,0.108064,0.188622,0.146133,0.725476,0.401889,0.045747,0.075156,0.062982,0.91191
min,-0.599400,0.000000,0.515000,0.000000,-1.000000,-0.771300,0.000000,0.739000,0.000000,-1.00000
25%,-0.019300,0.000000,0.728250,0.000000,-0.250000,-0.267050,0.000000,0.852500,0.000000,-1.00000
50%,0.000000,0.000000,0.859500,0.000000,0.000000,0.000000,0.051500,0.913000,0.019000,0.00000
75%,0.057625,0.164250,1.000000,0.213750,0.250000,0.328950,0.075750,0.952500,0.085000,1.00000
max,0.361200,0.301000,1.000000,0.438000,1.000000,0.636900,0.152000,1.000000,0.187000,1.00000


In [17]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!
ethereum_en_df.describe()

,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
count,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.00000,20.00000,20.000000
mean,0.057060,0.048550,0.919750,0.031700,0.15000,0.190925,0.078750,0.89145,0.02975,0.300000
std,0.232918,0.082891,0.102651,0.083104,0.67082,0.401422,0.085383,0.10092,0.04006,0.801315
min,-0.381800,0.000000,0.698000,0.000000,-1.00000,-0.526700,0.000000,0.69200,0.00000,-1.000000
25%,0.000000,0.000000,0.863750,0.000000,0.00000,0.000000,0.000000,0.84100,0.00000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.00000,0.025800,0.066500,0.91050,0.00000,0.500000
75%,0.201725,0.097750,1.000000,0.000000,1.00000,0.514625,0.120500,1.00000,0.06125,1.000000
max,0.458800,0.250000,1.000000,0.302000,1.00000,0.834000,0.249000,1.00000,0.11500,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereium had the highest mean positive score at 0.078750

Q: Which coin had the highest compound score?

A: Ethereium had the highestest compound score at 0.834

Q. Which coin had the highest positive score?

A:  Ethereium had the highestest positive score of 0.249

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [18]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [41]:
# Instantiate the lemmatizer
# YOUR CODE HERE!
lemmatizer = WordNetLemmatizer()

In [47]:
s = ethereum_en_df['text'][0]
sent_tokenize(s)

['In February, shit hit the fan in the usual way: An old tweet resurfaced.',
 'Brantly Millegan, director of operations at Ethereum Name Service (ENS), a web3 business, had written the following in May 201… [+3096 chars]']

In [48]:
[word_tokenize(t) for t in sent_tokenize(s)]

[['In',
  'February',
  ',',
  'shit',
  'hit',
  'the',
  'fan',
  'in',
  'the',
  'usual',
  'way',
  ':',
  'An',
  'old',
  'tweet',
  'resurfaced',
  '.'],
 ['Brantly',
  'Millegan',
  ',',
  'director',
  'of',
  'operations',
  'at',
  'Ethereum',
  'Name',
  'Service',
  '(',
  'ENS',
  ')',
  ',',
  'a',
  'web3',
  'business',
  ',',
  'had',
  'written',
  'the',
  'following',
  'in',
  'May',
  '201…',
  '[',
  '+3096',
  'chars',
  ']']]

In [25]:
# Create a list of stopwords
# YOUR CODE HERE!
stopwords = ('said', 'sent', 'found', 'including', 'today', 'announced', 'week', 'basically', 'also')


# Expand the default stopwords list if necessary
# YOUR CODE HERE!

stopwords_expanded = ()

In [55]:
article = ethereum_en_df['text']
article

0     In February, shit hit the fan in the usual way...
1     Coinbase reported that the share of trading vo...
2     Illustration by James Bareham / The Verge\r\n\...
3     If it sounds too good to be true, youre not wr...
4     It seems that in 2022, you cant escape from th...
5     The cryptocurrency boom over the past few year...
6     A version of this article was published in TIM...
7     Joe Biden is dipping his toes into crypto. On ...
8     The cryptocurrency boom over the past few year...
9     The cryptocurrency boom over the past few year...
10    If blockchain technology is to reach true mass...
11    The news: Blockchain specialist ConsenSys scoo...
12    (Kitco News) The escalating Russia-Ukraine cri...
13    During the days leading up to the March 11 pur...
14    The travel services industry and fast-food cha...
15    Personal Finance Insider writes about products...
16    A Ukrainian nonprofit has raised bitcoin worth...
17    As the international community rallies aro

In [18]:
# Complete the tokenizer function
def tokenizer(text):
    
    """Tokenizes text."""
    

    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root word
   
    # Convert the words to lowercase
    
    # Remove the stop words
    
    return tokens

In [52]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

AttributeError: 'tuple' object has no attribute 'words'

In [20]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [21]:
from collections import Counter
from nltk import ngrams

In [22]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [24]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [25]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [26]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [27]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [28]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [29]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [30]:
import spacy
from spacy import displacy

In [31]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [32]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [33]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [34]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [35]:
# Render the visualization
# YOUR CODE HERE!

In [36]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [37]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [38]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [39]:
# Render the visualization
# YOUR CODE HERE!

In [40]:
# List all Entities
# YOUR CODE HERE!

---